# Data Preprocessing

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%cd /content/drive/MyDrive/MSC_Intro_to_NLP_Group_Project/

/content/drive/MyDrive/MSC_Intro_to_NLP_Group_Project


In [ ]:
!pip install nltk
!pip install gensim
!pip install 'transformers[torch]'
!pip install datasets
!pip install tensorflow
!pip install torch
!pip install contractions


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 270.9/270.9 kB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 10.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 289.9/289.9 kB 2.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.8/110.8 kB 5.8 MB/s eta 0:00:00


In [ ]:
import json
from nltk.tokenize import word_tokenize
from gensim.models import Word2Vec
import numpy as np
import pandas as pd
from transformers import AutoTokenizer
from datasets import load_dataset
import torch
import torch.nn as nn
import torch.optim as optim
import pandas as pd
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from tqdm import tqdm
import os
import re
import nltk
import string
from nltk import word_tokenize, pos_tag
import contractions
import pandas as pd
import spacy
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
!python -m spacy download en_core_web_sm

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


2024-01-21 23:41:12.285850: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-01-21 23:41:12.285916: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-01-21 23:41:12.287189: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-01-21 23:41:13.653385: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 27.4 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


In [ ]:
# Using the GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)
# gpu_name = torch.cuda.get_device_name(device)
# print(gpu_name)

cpu


RuntimeError: Found no NVIDIA driver on your system. Please check that you have an NVIDIA GPU and installed a driver from http://www.nvidia.com/Download/index.aspx

In [ ]:
# Load spaCy model outside of the function to avoid reloading it each time the function is called
nlp = spacy.load("en_core_web_sm")

In [ ]:
# Function to capitalize the first letter of each sentence and proper nouns
def capitalize_first_and_proper_nouns(text):
    # Process the text using spaCy to create a Doc object
    doc = nlp(text)

    result = []

    # Iterate over the sentences in the Doc
    for sent in doc.sents:
        # Iterate over the tokens in the sentence
        for token in sent:
            # Capitalize the first letter of each sentence and proper nouns
            if token.is_sent_start or token.pos_ == 'PROPN':
                result.append(token.text.capitalize())
            else:
                result.append(token.text)

    # Rejoin the tokens into a single string
    return ' '.join(result)

# Function to remove repeated punctuations
def remove_repeated_punctuations(sentence):
    # Use regular expression to remove consecutive repeated punctuations
    cleaned_sentence = re.sub(r'(\W)\1+', r'\1', sentence)
    return cleaned_sentence

# Function to expand contractions
def expand_contractions(text):
    return contractions.fix(text)

# Define a tokenization function
def tokenize_sentences(sentences):
    return [word_tokenize(sentence) for sentence in sentences]

def fix_general_spacing(sentence):
    # Fix space before punctuation (like ' ,' to ',')
    sentence = re.sub(r'\s([,.?!:;])', r'\1', sentence)
    # Fix space after punctuation (like ' . ' to '. ')
    sentence = re.sub(r'([,.?!:;])\s', r'\1 ', sentence)
    # Fix space in contractions (like "don 't" to "don't")
    sentence = re.sub(r"\b(\w+)\s('t|'s|'m|'ll|'ve|'re|'d|n't)\b", r"\1\2", sentence)
    # Reduce multiple spaces between words to a single space
    sentence = re.sub(r'\s{2,}', ' ', sentence)
    return sentence

def preprocess(text):
    # text = text.lower()
    text = expand_contractions(text)
    text = remove_repeated_punctuations(text)
    text = capitalize_first_and_proper_nouns(text)
    text = fix_general_spacing(text)
    return text


In [ ]:
# # Load the preprocessed data from the JSON file
# data_files={
#     "train":"data_train.json",
# }

# dataset = load_dataset("json", data_files=data_files)
# print(dataset)

json_file_path = "data_train.json"

with open(json_file_path, 'r') as file:
    dataset = json.load(file)


# a dataset class to load and preprocess the data

In [ ]:
class CustomDataset(Dataset):
    def __init__(self, data, transform=None):
        self.data = data
        self.transform = transform

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        item = self.data[idx]
        return {
            "id": item["id"],
            "topic": item["topic"],
            "input_seq": torch.tensor(item["input_seq"]),  # Make sure this key matches the one used in numericalize_data
            "output_seq": torch.tensor(item["output_seq"]),  # Make sure this key matches the one used in numericalize_data
        }


Creating a Bi-Directional RNN Model

In [ ]:
def preprocess_data(data):
    for item in data:
        item["transformation"]["informal"] = preprocess(item["transformation"]["informal"])
    return data

In [ ]:
# preprocessed_data = preprocess_data(dataset)


In [ ]:
def tokenize_data(data):
    for item in data:
        item["tokenized_informal"] = word_tokenize(item["transformation"]["informal"])
        item["tokenized_formal"] = word_tokenize(item["transformation"]["formal.ref0"])
    return data


In [ ]:
def build_vocab(data):
    input_vocab = set()
    output_vocab = set()

    for item in data:
        input_vocab.update(item["tokenized_informal"])
        output_vocab.update(item["tokenized_formal"])

    input_vocab = {word: idx for idx, word in enumerate(input_vocab)}
    output_vocab = {word: idx for idx, word in enumerate(output_vocab)}

    return input_vocab, output_vocab


In [ ]:
def pad_sequence(sequence, max_length, vocab):
    padded_sequence = [vocab[word] for word in sequence if word in vocab]
    padded_sequence += [0] * (max_length - len(padded_sequence))
    return padded_sequence

def numericalize_data(data, input_vocab, output_vocab):
#     max_input_length = max(len(item["tokenized_informal"]) for item in data)
#     max_output_length = max(len(item["tokenized_formal"]) for item in data)
    max_input_length = 65
    max_output_length = 65

    for item in data:
        item["input_seq"] = pad_sequence(item["tokenized_informal"], max_input_length, input_vocab)
        item["output_seq"] = pad_sequence(item["tokenized_formal"], max_output_length, output_vocab)

    return data


Defining a Bi-Directional RNN Model

In [ ]:
class BiRNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(BiRNN, self).__init__()
        self.embedding = nn.Embedding(input_size, hidden_size)
        self.birnn = nn.LSTM(hidden_size, hidden_size, bidirectional=True)
        self.fc = nn.Linear(hidden_size * 2, len(output_vocab))

    def forward(self, x):
        embedded = self.embedding(x)
        output, _ = self.birnn(embedded)
        output = self.fc(output)
        return torch.nn.functional.log_softmax(output, dim=-1)


Training function

In [ ]:
def train_model(model, train_loader, criterion, optimizer, num_epochs=30):
    model.train()
    for epoch in range(num_epochs):
        total_loss = 0
        for batch in tqdm(train_loader, desc=f"Epoch {epoch + 1}/{num_epochs}"):
            inputs = batch["input_seq"].to(device)
            targets = batch["output_seq"].to(device)

            optimizer.zero_grad()

            outputs = model(inputs)
            loss = criterion(outputs.view(-1, len(output_vocab)), targets.view(-1))
            loss.backward()
            optimizer.step()

            total_loss += loss.item()

        print(f"Epoch {epoch + 1}, Loss: {total_loss / len(train_loader)}")


preparing the data

In [ ]:
data = preprocess_data(dataset)
data = tokenize_data(data)


In [ ]:
print(data[1])

In [ ]:
input_vocab, output_vocab = build_vocab(data)
data = numericalize_data(data, input_vocab, output_vocab)

In [ ]:
print(len(data[0]['input_seq']))
print(len(data))
print(data[0]['input_seq'])
print(len(data[0]['output_seq']))
print(data[0]['output_seq'])


In [ ]:
train_data = data
train_dataset = CustomDataset(train_data)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)


In [ ]:
model = BiRNN(len(input_vocab), 256, len(output_vocab)).to(device)
criterion = nn.CrossEntropyLoss(ignore_index=0)
optimizer = optim.Adam(model.parameters(), lr=0.001)


In [ ]:
trained_model = train_model(model, train_loader, criterion, optimizer)

In [ ]:
torch.save(model.state_dict(), 'BiRNN.pth')
# Save other information if needed (e.g., vocabulary)
torch.save({
    'input_vocab': input_vocab,
    'output_vocab': output_vocab,
}, 'BiRNN_model_info.pth')


In [ ]:
# Load model architecture
loaded_model = BiRNN(len(input_vocab), 256, len(output_vocab)).to(device)

# Load the trained weights
loaded_model.load_state_dict(torch.load('BiRNN.pth', map_location=device))

# Set the model to evaluation mode
loaded_model.eval()

# Load other information if needed
model_info = torch.load('BiRNN_model_info.pth')
input_vocab = model_info['input_vocab']
output_vocab = model_info['output_vocab']

In [ ]:
def predict(model, input_sentence, input_vocab, output_vocab):
    # Preprocess and tokenize the input sentence
    preprocessed_input = preprocess(input_sentence)
    tokenized_input = word_tokenize(preprocessed_input)

    # Numericalize the input sequence
    numericalized_input = pad_sequence(tokenized_input, max_input_length, input_vocab)
    numericalized_input = torch.tensor([numericalized_input]).to(device)

    # Make the prediction
    with torch.no_grad():
        model_output = model(numericalized_input)

    # Get the predicted indices
    _, predicted_indices = torch.max(model_output, dim=-1)
    predicted_indices_list = predicted_indices.tolist()


    # Convert indices back to words in the output vocabulary
    predicted_words = [word for idx, word in output_vocab.items() if idx in predicted_indices_list]

    # Return the predicted words
    return predicted_words

# Example usage:
input_sentence = "Hey, You're doing it wrong!!!"
predictions = predict(loaded_model, input_sentence, input_vocab, output_vocab)
print("Predicted output:", predictions)


In [ ]:
# Save the entire model
torch.save(model.state_dict(), 'bi_rnn_model.pth')

# Save the vocabularies as well for future reference
vocabularies = {
    'input_vocab': input_vocab,
    'output_vocab': output_vocab
}
with open('vocabularies.json', 'w') as vocab_file:
    json.dump(vocabularies, vocab_file)


In [ ]:
# Load the vocabularies
with open('vocabularies.json', 'r') as vocab_file:
    loaded_vocabularies = json.load(vocab_file)

input_vocab = loaded_vocabularies['input_vocab']
output_vocab = loaded_vocabularies['output_vocab']

# Create a new instance of the BiRNN model
loaded_model = BiRNN(len(input_vocab), 256, len(output_vocab)).to(device)

# Load the trained weights
loaded_model.load_state_dict(torch.load('bi_rnn_model.pth'))
loaded_model.eval()  # Set the model to evaluation mode

def predict(model, input_sequence, input_vocab, output_vocab):
    # Preprocess and tokenize input_sequence
    preprocessed_input = preprocess(input_sequence)
    tokenized_input = word_tokenize(preprocessed_input)

    # Numericalize input using the loaded vocabularies
    numericalized_input = pad_sequence(tokenized_input, max_input_length, input_vocab)

    # Convert numericalized input to PyTorch tensor
    input_tensor = torch.tensor(numericalized_input).unsqueeze(0).to(device)

    # Make prediction using the loaded model
    with torch.no_grad():
        model_output = model(input_tensor)

    # Process the model output
    _, predicted_indices = torch.max(model_output, dim=2 )

    # Convert predicted indices to words using the output vocabulary
    predicted_words = [word for index in predicted_indices.squeeze().tolist() for word, idx in output_vocab.items() if idx == index]

    # Join the words into a string (or use as needed)
    processed_output = ' '.join(predicted_words)

    return processed_output


In [ ]:
# Example usage:
input_sentence = "I'm doing great and studies're also great!!!"
predictions = predict(loaded_model, input_sentence, input_vocab, output_vocab)
print("Predicted output:", predictions)

In [ ]:
def preprocess_unseen_data(text):
    # Apply the same preprocessing steps as during training
    text = preprocess(text)
    tokenized_text = word_tokenize(text)
    padded_sequence = pad_sequence(tokenized_text, max_input_length, input_vocab)
    return torch.tensor(padded_sequence).unsqueeze(0)  # Add a batch dimension

unseen_data = "Yo! What's up??"
preprocessed_data = preprocess_unseen_data(unseen_data)

# Move the data to the appropriate device (CPU or GPU)
preprocessed_data = preprocessed_data.to(device)

# Make predictions
with torch.no_grad():
    model_output = loaded_model(preprocessed_data)

# Process the model output as needed
# For example, you can convert the output to probabilities and extract the predicted sequence
probabilities = torch.nn.functional.softmax(model_output, dim=-1)
predicted_sequence_indices = torch.argmax(probabilities, dim=-1).squeeze().tolist()

# Convert indices back to words using the output vocabulary
predicted_sequence_words = [word for idx, word in output_vocab.items() if idx in predicted_sequence_indices]
predicted_sequence_text = ' '.join(predicted_sequence_words)

print("Predicted Sequence:", predicted_sequence_text)


In [ ]:


# tokenizer = AutoTokenizer.from_pretrained('bert-base-cased')

# # Tokenize the informal sentences
# def preprocess_function(examples, input_field="informal", target_field="formal.ref0"):
#     inputs = [ex[input_field] for ex in examples["transformation"]]
#     targets = [ex[target_field] for ex in examples["transformation"]]

#     new_examples = tokenizer(
#         inputs, text_target=targets, max_length=64, truncation=True, padding="max_length"
#     )

#     return new_examples

# def create_multi_ref_dataset(dataset):
#   for i, target_field in enumerate(['formal.ref0', 'formal.ref1', 'formal.ref2', 'formal.ref3', 'rule_based_preprocessed']):
#     new_dataset = preprocess_function(dataset, 'informal', target_field)
#     dataset = dataset.add_column(f'labels_{i}', new_dataset['labels'])
#     if i == 0:
#       dataset = dataset.add_column('input_ids', new_dataset['input_ids'])
#       dataset = dataset.add_column('token_type_ids', new_dataset['token_type_ids'])
#       dataset = dataset.add_column('attention_mask', new_dataset['attention_mask'])

#   return dataset

# train_dataset = dataset['train'].map(
#     preprocess_function,
#     batched=True,
# )

# # tokenizer.convert_ids_to_tokens(train_dataset['input_ids'][0])


In [ ]:
print(train_dataset['input_ids'][0])